# 1. [30pt] Implement the Quantum Teleportation Protocol 

In [22]:
import pennylane as qml
import numpy as np

dev = qml.device("default.qubit", wires=3)

def psi_preparing(alpha, beta):
    norm = (abs(alpha)**2 + abs(beta)**2)**0.5
    alpha, beta = alpha / norm, beta / norm
    qml.QubitStateVector(np.array([alpha, beta]), wires=0)


@qml.qnode(dev)
def teleportation(alpha, beta):
    psi_preparing(alpha, beta)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=0)
    m0 = qml.measure(0)
    m1 = qml.measure(1)
    qml.cond(m1, qml.PauliX)(wires=2)
    qml.cond(m0, qml.PauliZ)(wires=2)
    return qml.probs(wires=[2])


alpha = 1 / (3**0.5)
beta = (2 / 3)**0.5

probs = teleportation(alpha, beta)
print("Measurement probabilities of Bob's qubit:")
print("P(|0>) =", probs[0])
print("P(|1>) =", probs[1])


Measurement probabilities of Bob's qubit:
P(|0>) = 0.3333333333333333
P(|1>) = 0.6666666666666665


# 2. [30pt] Violate the CHSH inequality using VQE

In [27]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=2)

def bell_state():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])

def measure(theta, phi, wire):
    qml.RY(theta, wires=wire)
    qml.RZ(phi, wires=wire)

@qml.qnode(dev, interface="autograd")
def E_a0b0(params):
    theta_a0, phi_a0, _, _, theta_b0, phi_b0, _, _ = params
    bell_state()
    measure(theta_a0, phi_a0, 0)
    measure(theta_b0, phi_b0, 1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

@qml.qnode(dev, interface="autograd")
def E_a0b1(params):
    theta_a0, phi_a0, _, _, _, _, theta_b1, phi_b1 = params
    bell_state()
    measure(theta_a0, phi_a0, 0)
    measure(theta_b1, phi_b1, 1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

@qml.qnode(dev, interface="autograd")
def E_a1b0(params):
    _, _, theta_a1, phi_a1, theta_b0, phi_b0, _, _ = params
    bell_state()
    measure(theta_a1, phi_a1, 0)
    measure(theta_b0, phi_b0, 1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

@qml.qnode(dev, interface="autograd")
def E_a1b1(params):
    _, _, theta_a1, phi_a1, _, _, theta_b1, phi_b1 = params
    bell_state()
    measure(theta_a1, phi_a1, 0)
    measure(theta_b1, phi_b1, 1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

def CHSH(params):
    return E_a0b0(params) + E_a0b1(params) + E_a1b0(params) - E_a1b1(params)



opt = qml.GradientDescentOptimizer(stepsize=0.4)

pie = 3.1416

params = np.array([0.0,0.0,pie/2, 0.0,pie/4, 0.0,pie /4,0.0], requires_grad=True)

for i in range(100):
    params, prev_cost = opt.step_and_cost(lambda x: -CHSH(x), params)
    val = CHSH(params)
    if i % 10 == 0:
        print(f"Step {i}: CHSH = {val}")
    if np.abs(val - prev_cost) < 1e-4:
        print(f"Converged at step {i}")
        break

print("Final CHSH value:", val)
print("Optimized params:", params)



Step 0: CHSH = 2.405864903166582
Step 10: CHSH = 2.8284270253416675
Step 20: CHSH = 2.8284271247461836
Step 30: CHSH = 2.828427124746189
Step 40: CHSH = 2.82842712474619
Step 50: CHSH = 2.8284271247461903
Step 60: CHSH = 2.8284271247461903
Step 70: CHSH = 2.8284271247461903
Step 80: CHSH = 2.8284271247461903
Step 90: CHSH = 2.8284271247461903
Final CHSH value: 2.8284271247461903
Optimized params: [ 0.39270092  0.          1.96349725  0.          1.17809908  0.
 -0.39269725  0.        ]


In [28]:
# If we check
print(2*(2**0.5))


2.8284271247461903


# 3. [40pt]  Generating arbitrary quantum superposition states 

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import math

def create_superposition_N(N):
    n = math.ceil(math.log2(N)) 
    dev = qml.device("default.qubit", wires=n)

    @qml.qnode(dev)
    def circuit():
        for i in range(n):
            qml.Hadamard(wires=i)
        return qml.probs(wires=range(n))

    probs = circuit()
    state_indices = list(range(2**n))

    valid_probs = probs[:N]
    total = np.sum(valid_probs)

    normalized_probs = valid_probs / total

    print(f"N = {N} -> using {n} qubits")
    for i in range(N):
        b = format(i, f"0{n}b")
        print(f"(|{b}>): amplitude tends to {normalized_probs[i]**2}")
        

create_superposition_N(5)
create_superposition_N(7)
create_superposition_N(31)



N = 5 -> using 3 qubits
(|000>): amplitude tends to 0.039999999999999994
(|001>): amplitude tends to 0.039999999999999994
(|010>): amplitude tends to 0.039999999999999994
(|011>): amplitude tends to 0.039999999999999994
(|100>): amplitude tends to 0.039999999999999994
N = 7 -> using 3 qubits
(|000>): amplitude tends to 0.020408163265306114
(|001>): amplitude tends to 0.020408163265306114
(|010>): amplitude tends to 0.020408163265306114
(|011>): amplitude tends to 0.020408163265306114
(|100>): amplitude tends to 0.020408163265306114
(|101>): amplitude tends to 0.020408163265306114
(|110>): amplitude tends to 0.020408163265306114
N = 31 -> using 5 qubits
(|00000>): amplitude tends to 0.0010405827263267424
(|00001>): amplitude tends to 0.0010405827263267424
(|00010>): amplitude tends to 0.0010405827263267424
(|00011>): amplitude tends to 0.0010405827263267424
(|00100>): amplitude tends to 0.0010405827263267424
(|00101>): amplitude tends to 0.0010405827263267424
(|00110>): amplitude tends 